## Tutorial number 1
### Working with ethoscope data
The tutorial will guide you through loading data, visualisation and plotting using the common sleep functions

## 1. Load the dummy dataset

In [1]:
import pandas as pd
import numpy as np 

import ethoscopy as etho

In [2]:
# This tutorial requires version 1.1.7 or greater
etho.__version__

'2.0.dev0'

In [3]:
df1 = pd.read_pickle('/home/lab/Desktop/test_ago_hmm/pickles/air_1_data_c.pkl')
df2 = pd.read_pickle('/home/lab/Desktop/test_ago_hmm/pickles/air_2_data_c.pkl')
pdf1 = pd.read_pickle('/home/lab/Desktop/test_ago_hmm/pickles/air_1_puff_c.pkl')
pdf2 = pd.read_pickle('/home/lab/Desktop/test_ago_hmm/pickles/air_2_puff_c.pkl')
df = etho.concat(df1, df2)
pdf = etho.concat(pdf1, pdf2)
df = etho.behavpy(df, df.meta, check = True, canvas = 'seaborn')
pdfs = etho.behavpy(pdf, pdf.meta, check = True, canvas = 'seaborn')
pdfp = etho.behavpy(pdf, pdf.meta, check = True, canvas = 'plotly')

In [4]:
# df = pd.read_pickle('/home/lab/Desktop/test_ago_hmm/pickles/acv_response_data.pkl')
# pdf = pd.read_pickle('/home/lab/Desktop/test_ago_hmm/pickles/acv_response_puff.pkl')
# df = etho.behavpy(df, df.meta)
# pdfs = etho.behavpy(pdf, pdf.meta, check = True, canvas = 'seaborn')
# pdfp = etho.behavpy(pdf, pdf.meta, check = True, canvas = 'plotly')

In [5]:
from ethoscopy.misc.get_HMM import get_HMM
h = get_HMM('M')

In [6]:
# pdfp = pdfp[pdfp['has_interacted'] == 1]
# pdfs = pdfs[pdfs['has_interacted'] == 1]

In [8]:
f, x = pdfs.plot_hmm_response(df, hmm = [h, h], t_bin = [60, 120])# facet_col='machine_name', facet_arg=['ETHOSCOPE_032', 'ETHOSCOPE_033']);

TypeError: HMM object must have decode() method

In [ ]:
f, x = pdfs.plot_hmm_response(df, hmm = h, facet_col='machine_name');

In [ ]:
f, x = pdfp.plot_hmm_response(df, hmm = h);#, facet_col='machine_name');
f

In [ ]:
f, x = pdfp.plot_hmm_response(df, hmm = h, facet_col='machine_name', col_uniform=True);
f

In [ ]:
f, x = pdfs.plot_hmm_response(df, hmm = h, facet_col='machine_name');

In [ ]:
# pdfs = pdfs[pdfs['has_interacted'] == 1]
pdfs.plot_habituation(plot_type = 'number', x_limit = 20, title = 'what', facet_col='machine_name');

In [ ]:
f = pdfs.plot_response_overtime(wrapped = True)#, facet_col='machine_name')
f.show()

In [ ]:
f = pdfp.plot_response_overtime(wrapped = True)#, facet_col='machine_name')
f.show()

In [ ]:
# pdfp.plot_response_over_hmm_bouts(df, hmm=h, variable = 'moving', response_col = 'has_responded')

In [ ]:
f = pdfs.plot_response_over_activity(df, activity = 'active', variable = 'moving', response_col = 'has_responded');#, facet_col = 'machine_name');
f.show()

In [ ]:
f = pdfp.plot_response_over_activity(df, activity = 'both', variable = 'moving', response_col = 'has_responded', facet_col = 'machine_name');
f.show()

In [ ]:
f = pdfp.plot_response_over_hmm_bouts(df, hmm=h, variable = 'moving', response_col = 'has_responded', labels = None, colours =  ['darkgreen', 'green', 'orange', 'yellow'])#, x_limit = 30, t_bin = 60, func = 'max', title = '', grids = False, t_column = 't', figsize = (0,0)):
f


In [ ]:
# pdfs = pdfs[pdfs['has_interacted'] == 1]
pdfs.plot_habituation(plot_type = 'number', x_limit = 20, title = 'what', facet_col='machine_name');

In [ ]:
f, x = pdfs.plot_response_quantify(facet_col='machine_name')

In [ ]:
f, x = pdfp.plot_response_quantify(facet_col='machine_name', title = 'what')
f.show()

In [ ]:
f, x = pdfs.plot_response_quantify()

In [ ]:
f, x = pdfp.plot_response_quantify()
f.show()

In [ ]:
from ethoscopy.misc.get_HMM import get_HMM
h = get_HMM('M')

In [ ]:
pdfs.plot_hmm_response(df, hmm = h, facet_col='machine_name')

In [ ]:
# df1 = pd.read_pickle('/home/lab/Desktop/test_ago_hmm/pickles/air_1_data_c.pkl')
# df2 = pd.read_pickle('/home/lab/Desktop/test_ago_hmm/pickles/air_2_data_c.pkl')
# pdf1 = pd.read_pickle('/home/lab/Desktop/test_ago_hmm/pickles/air_1_puff_c.pkl')
# pdf2 = pd.read_pickle('/home/lab/Desktop/test_ago_hmm/pickles/air_2_puff_c.pkl')
# df = etho.concat(df1, df2)
# pdf = etho.concat(pdf1, pdf2)
# df = etho.behavpy(df, df.meta, check = True, canvas = 'seaborn')
# pdfs = etho.behavpy(pdf, pdf.meta, check = True, canvas = 'seaborn')
# pdfp = etho.behavpy(pdf, pdf.meta, check = True, canvas = 'plotly')

In [ ]:
# tdf = df.xmv('date', '2024-03-14')
# tpdf = pdf.xmv('date', '2024-03-14')

In [ ]:
pdfs = pdfs[pdfs['has_interacted'] == 1]
pdfs.plot_habituation(plot_type = 'number', x_limit = 20, title = 'what', facet_col='machine_name');

In [ ]:
x = pdfp.plot_habituation(plot_type = 'number', x_limit = 20, title = 'what', facet_col='machine_name');
x.show()

In [ ]:
for i

In [ ]:
pdfs.plot_response_overtime(t_bin_hours=1, wrapped = True, facet_col = 'type');

In [ ]:
# fig = pdfp.plot_response_overtime(t_bin_hours=1, wrapped = True, facet_col = 'type')
# fig.show()

In [ ]:
# tpdf = tpdf[tpdf['has_interacted'] == 1]

In [ ]:
# x = pdfs.plot_response_over_activity(df, activity = 'inactive', variable = 'moving', facet_col = 'machine_name', facet_arg = None, facet_labels = None, title = '', t_column = 't', grids = False);

In [ ]:
x = pdfp.plot_response_over_activity(df, activity = 'inactive', variable = 'moving', facet_col = 'machine_name', facet_arg = None, facet_labels = None, title = '', t_column = 't', grids = False)
x.show()

In [ ]:
# tdf = etho.behavpy(d, m, check = True, canvas = 'plotly')
# tdf.meta['exp'] = tdf.meta['odour'] + '-' + tdf.meta['rest']
# fig, stats_s = tdf.plot_response_quantify(title = '')
# fig.show()

In [ ]:
# fig, stats_s = tdf.plot_quantify('x', facet_col = 'exp')
# fig.show()

In [ ]:
# fig, stats_s = df.plot_quantify('x', facet_col = 'exp')

In [ ]:
ndf = df[df['has_interacted'] == 1]

In [ ]:
fig, stats_s = df.plot_response_quantify(title = '')
# fig.show()

In [ ]:
['t', *['t', 't']]

In [ ]:
fig, stats_s = df.plot_response_quantify(facet_col = 'exp', facet_arg = fargs, title = '')

In [ ]:
for i

In [ ]:
stats_s

In [ ]:
stats_s[stats_s['exp'] == 'air-first day'][stats_s[stats_s['exp'] == 'air-first day']['has_interacted'] == 'True Stimulus']

In [ ]:
df = etho.behavpy(d, m, check = True, canvas = 'plotly')
df.meta['exp'] = df.meta['odour'] + '-' + df.meta['rest']
fig, stats_p = df.plot_response_quantify(facet_col = 'exp', facet_arg = fargs, title = '')
fig.show()